In [1]:
import kfp
from kfp import dsl
import kfp.components as components
from minio import Minio
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [7]:
torch.tensor([[6.8,  2.8,  4.8,  1.4], [6.0,  3.4,  4.5,  1.6]]

torch.Tensor

In [36]:
minio_client = Minio(
    "192.168.0.5:9000",
    access_key="minio",
    secret_key="minio123",
    secure=False
)
minio_bucket = "test"

iris = load_iris()
X = iris['data']
y = iris['target']

df_X = pd.DataFrame(X)
df_y = pd.DataFrame(y)

df_X.to_csv("/tmp/iris_df_X.csv", index=False)
minio_client.fput_object(minio_bucket,"iris_df_X.csv","/tmp/iris_df_X.csv")
df_y.to_csv("/tmp/iris_df_y.csv", index=False)
minio_client.fput_object(minio_bucket,"iris_df_y.csv","/tmp/iris_df_y.csv")

('bef8f986f3777d21f049471242c7fd26-1', None)

In [ ]:
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50)
        self.layer2 = nn.Linear(50, 50)
        self.layer3 = nn.Linear(50, 3)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1)
        return x

In [38]:
minio_client = Minio(
    "192.168.0.5:9000",
    access_key="minio",
    secret_key="minio123",
    secure=False
)
minio_bucket = "test"

minio_client.fget_object(minio_bucket,"iris_df_X.csv","/tmp/iris_df_X.csv")
minio_client.fget_object(minio_bucket,"iris_df_y.csv","/tmp/iris_df_y.csv")

df_X = pd.read_csv("/tmp/iris_df_X.csv")
df_y = pd.read_csv("/tmp/iris_df_y.csv")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

model     = Model(X_train.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn   = nn.CrossEntropyLoss()

print(model)

EPOCHS  = 100
loss_list     = np.zeros((EPOCHS,))
accuracy_list = np.zeros((EPOCHS,))

for epoch in range(EPOCHS):
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    loss_list[epoch] = loss.item()
    
    # Zero gradients
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    with torch.no_grad():
        y_pred = model(X_test)
        correct = (torch.argmax(y_pred, dim=1) == y_test).type(torch.FloatTensor)
        accuracy_list[epoch] = correct.mean()

torch.Size([120, 4])
torch.Size([30, 4])
torch.Size([120])
torch.Size([30])
Model(
  (layer1): Linear(in_features=4, out_features=50, bias=True)
  (layer2): Linear(in_features=50, out_features=50, bias=True)
  (layer3): Linear(in_features=50, out_features=3, bias=True)
)


In [42]:
# predict with trained model
model(torch.FloatTensor([[6.8,  2.8,  4.8,  1.4], [6.0,  3.4,  4.5,  1.6]]))

tensor([[0.0136, 0.6330, 0.3534],
        [0.0333, 0.6532, 0.3135]], grad_fn=<SoftmaxBackward>)

In [25]:
model.save

array([0.26666668, 0.26666668, 0.26666668, 0.26666668, 0.26666668,
       0.26666668, 0.26666668, 0.26666668, 0.26666668, 0.26666668,
       0.26666668, 0.26666668, 0.26666668, 0.26666668, 0.26666668,
       0.26666668, 0.26666668, 0.26666668, 0.26666668, 0.26666668,
       0.26666668, 0.26666668, 0.40000001, 0.53333336, 0.63333333,
       0.69999999, 0.73333335, 0.73333335, 0.73333335, 0.73333335,
       0.73333335, 0.73333335, 0.73333335, 0.73333335, 0.73333335,
       0.73333335, 0.73333335, 0.73333335, 0.73333335, 0.73333335,
       0.73333335, 0.73333335, 0.73333335, 0.73333335, 0.73333335,
       0.73333335, 0.73333335, 0.73333335, 0.73333335, 0.73333335,
       0.73333335, 0.73333335, 0.73333335, 0.73333335, 0.73333335,
       0.73333335, 0.73333335, 0.73333335, 0.73333335, 0.73333335,
       0.73333335, 0.73333335, 0.73333335, 0.73333335, 0.73333335,
       0.73333335, 0.73333335, 0.73333335, 0.73333335, 0.73333335,
       0.73333335, 0.73333335, 0.73333335, 0.73333335, 0.73333